# Hello AI PC 

This is a simple Jupyter Notebook to check device configuration

## Imports

We'll use the following imports in our example. `torch` and `torch_nn` are used for building and running ML models. We'll use them to define a small neural network and to generate the model weights. `os` is used for interacting with the operating system and is used to manage our environment variables, file paths, and directories. `subprocess` allows us to retrieve the hardware information. `onnx` and `onnxruntime` are used to work with our model in the ONNX format and for running our inference. `vai_q_onnx` is part of the Vitis AI Quantizer for ONNX models. We use it to perform quantization, converting the model into an INT8 format that is optimized for the NPU.

In [1]:
import torch
import torch.nn as nn
import os
import subprocess
import onnxruntime
import numpy as np
import onnx
import shutil
from timeit import default_timer as timer

## Device Configuration
Version 1.3 of the Ryzen AI Software supports AMD processors codenamed Phoenix, Hawk Point, Strix, Strix Halo and Krackan Point. These processors can be found in the following Ryzen series:   
Ryzen 200 Series   
Ryzen 7000 Series, Ryzen PRO 7000 Series   
Ryzen 8000 Series, Ryzen PRO 8000 Series   
Ryzen AI 300 Series, Ryzen AI PRO Series, Ryzen AI Max 300 Series   
   
For a complete list of supported devices, refer to the processor specifications page (look for the “AMD Ryzen AI” column towards the right side of the table, and select   “Available” from the pull-down menu).   
   
The rest of this document will refer to Phoenix as PHX, Hawk Point as HPT, Strix and Strix Halo as STX, and Krackan Point as KRK.   

In [2]:
import platform
import sys
import cpuinfo
from pprint import pprint

print("Python version:", sys.version)
print("Platform:", platform.platform())
print("Processor Architecture:", platform.architecture())
print("Machine:", platform.machine())
print("System:", platform.system())
cpu_info = cpuinfo.get_cpu_info()
print("Processor:", cpu_info["brand_raw"])

Python version: 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
Platform: Windows-10-10.0.26100-SP0
Processor Architecture: ('64bit', 'WindowsPE')
Machine: AMD64
System: Windows
Processor: AMD Ryzen AI 9 HX 370 w/ Radeon 890M


## Check APU (NPU) type 
Useful to configure environment variables for hardware-specific optimization.

In [3]:
def get_apu_info():
    # Run pnputil as a subprocess to enumerate PCI devices
    command = r'pnputil /enum-devices /bus PCI /deviceids '
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    # Check for supported Hardware IDs
    apu_type = ''
    if 'PCI\\VEN_1022&DEV_1502&REV_00' in stdout.decode(): apu_type = 'PHX/HPT'
    if 'PCI\\VEN_1022&DEV_17F0&REV_00' in stdout.decode(): apu_type = 'STX'
    if 'PCI\\VEN_1022&DEV_17F0&REV_10' in stdout.decode(): apu_type = 'STX'
    if 'PCI\\VEN_1022&DEV_17F0&REV_11' in stdout.decode(): apu_type = 'STX'
    return apu_type

apu_type = get_apu_info()
print(f"APU Type: {apu_type}")

APU Type: STX


## Check Environment Variables

In [4]:
# XLNX_VART_FIRMWARE - Specifies the firmware file used by the NPU for runtime execution
# NUM_OF_DPU_RUNNERS - Specifies the number of DPU runners (processing cores) available for execution
# XLNX_TARGET_NAME - Name of the target hardware configuration

def set_environment_variable(apu_type):

    install_dir = os.environ['RYZEN_AI_INSTALLATION_PATH']
    match apu_type:
        case 'PHX/HPT':
            print("Setting environment for PHX/HPT")
            os.environ['XLNX_VART_FIRMWARE']= os.path.join(install_dir, 'voe-4.0-win_amd64', 'xclbins', 'phoenix', '1x4.xclbin')
            os.environ['NUM_OF_DPU_RUNNERS']='1'
            os.environ['XLNX_TARGET_NAME']='AMD_AIE2_Nx4_Overlay'
        case 'STX':
            print("Setting environment for STX")
            os.environ['XLNX_VART_FIRMWARE']= os.path.join(install_dir, 'voe-4.0-win_amd64', 'xclbins', 'strix', 'AMD_AIE2P_Nx4_Overlay.xclbin')
            os.environ['NUM_OF_DPU_RUNNERS']='1'
            os.environ['XLNX_TARGET_NAME']='AMD_AIE2_Nx4_Overlay'
        case _:
            print("Unrecognized APU type. Exiting.")
            exit()
    print('XLNX_VART_FIRMWARE=', os.environ['XLNX_VART_FIRMWARE'])
    print('NUM_OF_DPU_RUNNERS=', os.environ['NUM_OF_DPU_RUNNERS'])
    print('XLNX_TARGET_NAME=', os.environ['XLNX_TARGET_NAME'])

set_environment_variable(apu_type)

Setting environment for STX
XLNX_VART_FIRMWARE= C:\Program Files\RyzenAI\1.3.1\voe-4.0-win_amd64\xclbins\strix\AMD_AIE2P_Nx4_Overlay.xclbin
NUM_OF_DPU_RUNNERS= 1
XLNX_TARGET_NAME= AMD_AIE2_Nx4_Overlay
